In [ ]:
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt

import pymc3 as pm

print('Running on PyMC3 v{}'.format(pm.__version__))

Running on PyMC3 v3.7


#1.PyMCで確率分布に従う乱数を生成してみよう

確率分布の性質や推論の振る舞いを確認するために、乱数を生成した実験が欠かせません.

numpyやscipyなど、乱数生成に便利なライブラリは他にもありますが、練習のためにPyMCで生成してみましょう.

正規分布で生成してみます.

In [ ]:
with pm.Model() as model:
  x = pm.Normal('x')
  # パラメーターを指定しない場合 : mu=0, sd=1

作成した確率モデルからサンプリング

In [ ]:
with model:
  trace = pm.sample()


サンプリングの結果は trace オブジェクトに格納されています

In [ ]:
print(type(trace['x'])) # データ型
print(trace['x'].shape) # データサイズ

### 【演習】1.1 正規分布のパラメーターを指定して乱数を生成してみよう

平均と標準偏差を指定してみよう

In [ ]:
with pm.Model() as model:
  x = pm.Normal('x'
                , mu= # Enter here
                , sd= # Enter here
                )

with model:
  trace = pm.sample()

生成した 'x' の分布を確認

In [ ]:
plt.hist(trace['x'])

生成した'x'の平均と標準偏差は指定した値と近い結果になっているでしょうか？

In [ ]:
print('mean:', trace['x'].mean())
print('sd:', trace['x'].std())

### 【演習】1.2 指数分布に従う乱数を生成してみよう

ヒント：指数分布は 英語で Exponential Distribution です

[PyMCのリファレンス](https://docs.pymc.io/api/distributions/continuous.html)から探してみよう

In [ ]:
with pm.Model() as model:
  # Enter here

In [ ]:
with model:
  trace = pm.sample()

生成したデータの分布を確認してみよう

In [ ]:
# Enter here

生成したデータが、自分で指定したパラメーターと近い値になっているか確認してみよう

ヒント : λ = 1/E[x]

In [ ]:
# Enter here

### 【演習】1.3 二項分布のパラメーターを指定して乱数を生成してみよう

ヒント：二項分布は 英語で Binomial Distribution です

[PyMCのリファレンス](https://docs.pymc.io/api/distributions/discrete.html)から探してみよう

In [ ]:
with pm.Model() as model:
  x = pm.Binomial('x'
                  , n= # Enter here
                  , p= # Enter here
                  )

In [ ]:
with model:
  trace = pm.sample()

生成したデータの分布を確認してみよう

In [ ]:
x = range(trace['x'].max() + 1)
height = np.bincount(trace['x'])

plt.bar(x=x, height=height)

確率 を確認してみよう

ヒント : p = E[x] / n

In [ ]:
# Enter here

### 【演習】1.4 ポアソン分布に従う乱数を生成してみよう

ヒント：ポアソン分布は 英語で Poisson Distribution です

[PyMCのリファレンス](https://docs.pymc.io/api/distributions/discrete.html)から探してみよう

まずは確率モデルを定義してみよう

In [ ]:
with pm.Model() as model:
  # Enter here

In [ ]:
with model:
  trace = pm.sample()

生成したデータの分布を確認してみよう

In [ ]:
# Enter here

生成したデータが、自分が指定したパラメーターと近い値になっているか確認してみよう

ヒント:ヒント:μ = E[x]

In [ ]:
# Enter here

#2.正規分布のパラメーターを推定してみよう

正しく推定できるか実験するために、正規分布に従う乱数を生成します

ここで生成した乱数のパラメーターを推定できるか実験してみます

In [ ]:
norm_dist = stats.norm(loc=50, scale=10)
# loc:平均, scale:標準偏差
np.random.seed(1234)
x = norm_dist.rvs(100)
print(x.mean())
print(x.std())

分布を確認してみます

In [ ]:
plt.hist(x)

確率モデルを設計します

簡単のために、正規分布の「平均」だけを推定することを考えてみます

ここでは正規分布の「平均」もまた正規分布である、と仮定します

In [ ]:
with pm.Model() as model:
  mu = pm.Normal('mu', mu=0, sd=10) # 「mu = 平均」 が従う分布を設計
  pm_x = pm.Normal('pm_x', mu=mu, sd=10, observed=x) # observed に先ほど生成した乱数を与える

In [ ]:
with model:
  trace = pm.sample(
      draws=500, # サンプル数
      chains=1,
      random_seed=1234)

mu がどの様な分布になっているか確認してみましょう。

In [ ]:
print(trace['mu'].mean())

In [ ]:
plt.hist(trace['mu'])

事前に設計した 'mu' は 平均0, 標準偏差10だったはずですが、

事後分布の平均は約50、標準偏差は約1であることがわかります。

次に、事前分布('mu'の分布)のパラメーターを変更してもう一度実行してみましょう。

In [ ]:
with pm.Model() as model:
  mu = pm.Normal('mu', mu=0, sd=1) # sd を　１ に変更してみる
  pm_x = pm.Normal('pm_x', mu=mu, sd=10, observed=x)

In [ ]:
with model:
  trace = pm.sample(
      draws=500,
      chains=1,
      random_seed=1234)

In [ ]:
plt.hist(trace['mu'])

推定値を確認してみよう

In [ ]:
print(trace['mu'].mean())

事前に生成した　x の分布とはかけ離れた結果が得られました.

なぜでしょうか？

実はMCMCでは、尤度が大きくなるようパラメーターを探索しているのですが、勾配降下法や最尤推定とは異なり、***確率***的に探索を行い、探索結果そのものを事後分布として扱っています.

今回の事前分布は平均0, 標準偏差1の正規分布としているために 50 を生成する***確率***が非常に小さくなってしまい、かけ離れた結果になってしまいました.

このように、事前分布の設計はパラメーターの取りうる範囲を考えながら設計する必要があります.

### 【演習】2.1 事前分布のパラメーターを変更し結果を確認してみよう


In [ ]:
# パラメーターを自由に変更してみよう
with pm.Model() as model:
  mu = pm.Normal('mu'
                    , mu= # Enter here
                    , sd= # Enter here
                    )
  pm_x = pm.Normal('pm_x', mu=mu, sd=10, observed=x)

In [ ]:
with model:
  trace = pm.sample(
        draws=500,
        chains=1,
        random_seed=1234)

結果を確認してみよう

In [ ]:
plt.hist(trace['mu'])

推定値を確認してみよう

In [ ]:
print(trace['mu'].mean())

### 2.2 適切な事前分布を考えてみよう

平均は推定できたので、標準偏差についても推定してみましょう.

【クイズ】 2.2.1 正規分布の標準偏差の、最大値と最小値はそれぞれどのような値ですか？

ヒント：[PyMCのリファレンス](https://docs.pymc.io/api/distributions/continuous.html)

*   最大値:
*   最小値:

【クイズ】 2.2.2 最大値,最小値の条件を満たす確率分布を探して1つ挙げてみましょう.

その確率分布のパラメーターも確認しましょう.
*   確率分布の名称:
*   パラメーター:

### 【演習】 2.3 上記で探した確率分布で確率モデルを設計してみよう.

In [ ]:
with pm.Model() as model:
  mu = pm.Normal('mu'
                    , mu= # Enter here
                    , sd= # Enter here
                    )
  
  sd = # Enter here
  
  pm_x = pm.Normal('pm_x', mu=mu, sd=sd, observed=x)

In [ ]:
with model:
  trace = pm.sample(
        draws=500,
        chains=1,
        random_seed=1234)

In [ ]:
plt.hist(trace['sd'])

推定値を確認してみよう

In [ ]:
print(trace['sd'].mean())

# 3.様々な確率分布で実験してみよう

###  【演習】 3.1 指数分布のパラメーターを推定してみよう

まずは指数分布に従う乱数を生成してみよう

ヒント：[scipyのリファレンス](https://docs.scipy.org/doc/scipy/reference/stats.html)

In [ ]:
# Enter here

生成した乱数の分布を確認してみよう

In [ ]:
 # Enter here

推定するためのモデルを作ってみよう

ヒント：指数分布のパラメーターが取りうる値の範囲は?

In [ ]:
with pm.Model() as model:

  lam = # Enter here

  pm_x = pm.Exponential('pm_x', lam=lam, observed='自分で定義した変数名称')

In [ ]:
with model:
  trace = pm.sample(
        draws=500,
        chains=1,
        random_seed=1234)

事後分布のヒストグラムを確認してみよう

In [ ]:
# Enter here

推定したパラメーターが設計した値と近似しているか確認してみよう

ヒント:scipyの loc = 1 / lam

In [ ]:
# Enter here

###  【演習】 3.2 二項分布のパラメーターを推定してみよう

まずは二項分布に従う乱数を生成してみよう

ヒント：[scipyのリファレンス](https://docs.scipy.org/doc/scipy/reference/stats.html)

In [ ]:
# Enter here

生成した乱数の分布を確認してみよう

In [ ]:
# Enter here

推定するためのモデルを作ってみよう

ヒント：二項分布のパラメーターが取りうる値の範囲は?

In [ ]:
with pm.Model() as model:
  # Enter here

In [ ]:
with model:
  trace = pm.sample(
        draws=500,
        chains=1,
        random_seed=1234)

事後分布のヒストグラムを確認してみよう

In [ ]:
# Enter here

推定値を確認してみよう

In [ ]:
# Enter here

###  【演習】 3.3 ポアソン分布のパラメーターを推定してみよう

まずはポアソン分布に従う乱数を生成してみよう

ヒント：[scipyのリファレンス](https://docs.scipy.org/doc/scipy/reference/stats.html)

In [ ]:
# Enter here

生成した乱数の分布を確認してみよう

In [ ]:
# Enter here

推定するためのモデルを作ってみよう


In [ ]:
# Enter here

In [ ]:
# Enter here

事後分布のヒストグラムを確認してみよう

In [ ]:
# Enter here

推定値を確認してみよう

In [ ]:
# Enter here